In [6]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
from deap import base, creator, tools, algorithms
from sklearn.model_selection import train_test_split, KFold
import mlflow
import mlflow.sklearn
import os

In [7]:
# Funções auxiliares
def get_graph_from_location(location_point, dist=2000, network_type='drive'):
    G = ox.graph_from_point(location_point, dist=dist, network_type=network_type)
    return nx.convert_node_labels_to_integers(G)


In [8]:

def get_node_coords(G, node):
    return G.nodes[node]['x'], G.nodes[node]['y']


In [9]:

def calculate_route_length(G, route):
    length = 0
    for i in range(len(route) - 1):
        try:
            length += nx.shortest_path_length(G, source=route[i], target=route[i + 1], weight='length')
        except nx.NetworkXNoPath:
            length += np.inf  # Adiciona uma penalidade alta se não houver caminho
    return length


In [10]:

# Função de avaliação (fitness)
def fitness(individual):
    total_length = 0
    for route in individual:
        total_length += calculate_route_length(G, route)
    return total_length,


In [11]:

# Inicialização do Algoritmo Genético
def init_toolbox(G, num_vehicles, num_points, depot):
    toolbox = base.Toolbox()
    
    # Estrutura da solução: lista de listas de nós
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    def init_route():
        return [depot] + random.sample(list(G.nodes), num_points) + [depot]
    
    toolbox.register("attr_route", init_route)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_route, n=num_vehicles)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", fitness)
    
    return toolbox


In [12]:

# Visualização da solução
def plot_routes(G, graph_routes, depot, waypoints, iteration, location_index):
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    fig, ax = ox.plot_graph(G, show=False, close=False, node_size=0)
    
    for route, color, points in zip(graph_routes, colors, waypoints):
        route_nodes = [get_node_coords(G, node) for node in route]
        x, y = zip(*route_nodes)
        ax.plot(x, y, color)
        point_coords = [get_node_coords(G, point) for point in points]
        px, py = zip(*point_coords)
        ax.scatter(px, py, c=color, s=50, zorder=5)
    
    depot_coords = get_node_coords(G, depot)
    ax.scatter(*depot_coords, c='yellow', s=100, zorder=5)
    
    plt.title(f'Iteration {iteration}, Location {location_index}')
    plt.savefig(f'output/iteration_{iteration}_location_{location_index}.png')
    plt.close()


In [13]:

# Função para mudar constantemente o mapa para áreas urbanas
def get_random_urban_location():
    # Lista de pontos centrais de áreas urbanas (latitude, longitude)
    urban_locations = [
        (-16.681, -49.256),  # Goiânia
        (-23.5505, -46.6333),  # São Paulo
        (-22.9068, -43.1729),  # Rio de Janeiro
        (-30.0346, -51.2177),  # Porto Alegre
        (-15.7942, -47.8822),  # Brasília
        (-19.932, -43.937),  # Belo Horizonte
        (-25.4284, -49.2733),  # Curitiba
        (-12.9714, -38.5014),  # Salvador
        (-8.0476, -34.8770),  # Recife
        (-3.7172, -38.5434)  # Fortaleza
    ]
    return random.choice(urban_locations)


In [14]:

def run_genetic_algorithm(location_index, location_point):
    global G  # Para uso na função fitness
    
    population_size = 30  # Reduzido para limitar a carga computacional
    generations = 50  # Reduzido para limitar a carga computacional
    
    G = get_graph_from_location(location_point)
    
    depot = random.choice(list(G.nodes))
    num_vehicles = random.randint(3, 7)  # Randomizar número de veículos
    num_points = random.randint(5, 12)  # Randomizar número de pontos
    
    toolbox = init_toolbox(G, num_vehicles, num_points, depot)
    
    population = toolbox.population(n=population_size)
    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=generations, verbose=True)
    
    best_individual = tools.selBest(population, k=1)[0]
    graph_routes = [[depot] + route[1:-1] + [depot] for route in best_individual]
    
    return graph_routes, best_individual, depot


In [15]:


# Configurar MLflow
mlflow.set_experiment("Urban Vehicle Routing")


<Experiment: artifact_location='file:///home/pedro/Documents/BIA/p3/BD-AED2-final/mlruns/997793492087020297', creation_time=1721494515721, experiment_id='997793492087020297', last_update_time=1721494515721, lifecycle_stage='active', name='Urban Vehicle Routing', tags={}>

In [16]:

# Verifique se o diretório de saída existe e, se não, crie-o
if not os.path.exists('output'):
    os.makedirs('output')


In [17]:

location_points = [get_random_urban_location() for _ in range(10)]


In [18]:

for iteration, location_point in enumerate(location_points):
    with mlflow.start_run(run_name=f"Iteration_{iteration}"):
        graph_routes, best_individual, depot = run_genetic_algorithm(iteration, location_point)
        
        if graph_routes:
            plot_routes(G, graph_routes, depot, best_individual, iteration, location_point)
            
            # Registrar parâmetros e métricas no MLflow
            mlflow.log_param("iteration", iteration)
            mlflow.log_param("location", location_point)
            mlflow.log_param("num_vehicles", len(best_individual))
            mlflow.log_param("num_points", len(best_individual[0]) - 2)
            mlflow.log_metric("total_length", fitness(best_individual)[0])
            
            # Registrar o gráfico gerado
            mlflow.log_artifact(f'output/iteration_{iteration}_location_{location_point}.png')
        else:
            print(f"Nenhuma solução encontrada para a localização {location_point} na iteração {iteration}")

/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	30    
1  	26    
2  	25    
3  	21    
4  	20    
5  	22    
6  	20    
7  	26    
8  	13    
9  	19    
10 	19    
11 	21    
12 	26    
13 	23    
14 	19    
15 	22    
16 	18    
17 	29    
18 	20    
19 	24    
20 	21    
21 	24    
22 	26    
23 	28    
24 	24    
25 	30    
26 	25    
27 	23    
28 	26    
29 	16    
30 	14    
31 	19    
32 	18    
33 	26    
34 	28    
35 	19    
36 	23    
37 	24    
38 	21    
39 	21    
40 	20    
41 	26    
42 	20    
43 	24    
44 	24    
45 	24    
46 	26    
47 	22    
48 	17    
49 	20    
50 	18    


/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	30    
1  	18    
2  	14    
3  	29    
4  	19    
5  	23    
6  	25    
7  	24    
8  	25    
9  	25    
10 	23    
11 	24    
12 	23    
13 	22    
14 	25    
15 	22    
16 	25    
17 	15    
18 	24    
19 	25    
20 	27    
21 	21    
22 	21    
23 	20    
24 	19    
25 	19    
26 	25    
27 	25    
28 	26    
29 	15    
30 	19    
31 	26    
32 	20    
33 	22    
34 	24    
35 	16    
36 	27    
37 	20    
38 	23    
39 	26    
40 	28    
41 	16    
42 	20    
43 	24    
44 	25    
45 	24    
46 	24    
47 	19    
48 	18    
49 	21    
50 	18    


/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


...

gen	nevals
0  	30    
1  	23    
2  	23    
3  	24    
4  	18    
5  	22    
6  	28    
7  	22    
8  	21    
9  	22    
10 	21    
11 	22    
12 	24    
13 	18    
14 	20    
15 	19    
16 	25    
17 	22    
18 	22    
19 	22    
20 	22    
21 	26    
22 	23    
23 	24    
24 	19    
25 	20    
26 	21    
27 	22    
28 	22    
29 	25    
30 	21    
31 	22    
32 	28    
33 	22    
34 	22    
35 	23    
36 	25    
37 	28    
38 	16    
39 	24    
40 	21    
41 	28    
42 	16    
43 	21    
44 	21    
45 	27    
46 	18    
47 	22    
48 	25    
49 	26    
50 	21    


/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	30    
1  	24    
2  	28    
3  	21    
4  	24    
5  	26    
6  	27    
7  	20    
8  	26    
9  	24    
10 	23    
11 	15    
12 	19    
13 	25    
14 	22    
15 	21    
16 	22    
17 	24    
18 	20    
19 	30    
20 	26    
21 	20    
22 	27    
23 	17    
24 	19    
25 	20    
26 	24    
27 	27    
28 	24    
29 	16    
30 	23    
31 	25    
32 	25    
33 	23    
34 	20    
35 	21    
36 	26    
37 	18    
38 	27    
39 	21    
40 	21    
41 	25    
42 	20    
43 	19    
44 	28    
45 	26    
46 	27    
47 	23    
48 	19    
49 	17    
50 	24    


/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	30    
1  	26    
2  	18    
3  	23    
4  	24    
5  	19    
6  	21    
7  	27    
8  	28    
9  	20    
10 	21    
11 	25    
12 	25    
13 	26    
14 	21    
15 	26    
16 	24    
17 	17    
18 	19    
19 	21    
20 	21    
21 	27    
22 	24    
23 	24    
24 	19    
25 	24    
26 	21    
27 	18    
28 	23    
29 	20    
30 	25    
31 	25    
32 	23    
33 	25    
34 	27    
35 	24    
36 	20    
37 	21    
38 	29    
39 	21    
40 	18    
41 	27    
42 	22    
43 	23    
44 	26    
45 	19    
46 	26    
47 	25    
48 	29    
49 	22    
50 	22    


---

## Teste 2

In [50]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
from deap import base, creator, tools, algorithms
import mlflow
import mlflow.sklearn
import os
import pickle
import time


In [66]:

# Funções auxiliares
def get_graph_from_location(location_point, dist=1000, network_type='drive'):
    G = ox.graph_from_point(location_point, dist=dist, network_type=network_type)
    return nx.convert_node_labels_to_integers(G)

def get_node_coords(G, node):
    return G.nodes[node]['x'], G.nodes[node]['y']

def calculate_route_length(G, route):
    length = 0
    for i in range(len(route) - 1):
        try:
            length += nx.shortest_path_length(G, source=route[i], target=route[i + 1], weight='length')
        except nx.NetworkXNoPath:
            length += np.inf  # Adiciona uma penalidade alta se não houver caminho
    return length

# Função de avaliação (fitness)
def fitness(individual):
    total_length = 0
    for route in individual:
        total_length += calculate_route_length(G, route)
    return total_length,

# Inicialização do Algoritmo Genético
def init_toolbox(G, num_vehicles, num_points, depot):
    toolbox = base.Toolbox()
    
    # Estrutura da solução: lista de listas de nós
    if "FitnessMin" in creator.__dict__:
        del creator.FitnessMin
    if "Individual" in creator.__dict__:
        del creator.Individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    def init_route():
        return [depot] + random.sample(list(G.nodes), num_points) + [depot]
    
    toolbox.register("attr_route", init_route)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_route, n=num_vehicles)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", fitness)
    
    return toolbox

# Visualização da solução
def plot_routes(G, graph_routes, depot, waypoints, iteration, location_index):
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    fig, ax = ox.plot_graph(G, show=False, close=False, node_size=0)
    
    for route, color, points in zip(graph_routes, colors, waypoints):
        route_nodes = [get_node_coords(G, node) for node in route]
        x, y = zip(*route_nodes)
        ax.plot(x, y, color)
        point_coords = [get_node_coords(G, point) for point in points]
        px, py = zip(*point_coords)
        ax.scatter(px, py, c=color, s=50, zorder=5)
    
    depot_coords = get_node_coords(G, depot)
    ax.scatter(*depot_coords, c='yellow', s=100, zorder=5)
    
    plt.title(f'Iteration {iteration}, Location {location_index}')
    plt.savefig(f'output/iteration_{iteration}_location_{location_index}.png')
    plt.close()

# Função para mudar constantemente o mapa para áreas urbanas
def get_random_urban_location():
    # Lista de pontos centrais de áreas urbanas (latitude, longitude)
    urban_locations = [
        (-16.681, -49.256),  # Goiânia
        (-23.5505, -46.6333),  # São Paulo
        (-22.9068, -43.1729),  # Rio de Janeiro
        (-30.0346, -51.2177),  # Porto Alegre
        (-15.7942, -47.8822),  # Brasília
        (-19.932, -43.937),  # Belo Horizonte
        (-25.4284, -49.2733),  # Curitiba
        (-12.9714, -38.5014),  # Salvador
        (-8.0476, -34.8770),  # Recife
        (-3.7172, -38.5434)  # Fortaleza
    ]
    return random.choice(urban_locations)

def run_genetic_algorithm(location_index, location_point):
    global G  # Para uso na função fitness
    
    population_size = 30  # Reduzido para limitar a carga computacional
    generations = 50  # Reduzido para limitar a carga computacional
    
    G = get_graph_from_location(location_point)
    
    depot = random.choice(list(G.nodes))
    num_vehicles = random.randint(2, 5)  # Randomizar número de veículos
    num_points = random.randint(5, 10)  # Randomizar número de pontos
    
    toolbox = init_toolbox(G, num_vehicles, num_points, depot)
    
    population = toolbox.population(n=population_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + stats.fields

    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=generations, 
                        stats=stats, halloffame=hof, verbose=True)
    
    best_individual = hof[0]
    graph_routes = [[depot] + route[1:-1] + [depot] for route in best_individual]
    
    return graph_routes, best_individual, depot

def plot_fitness(logbook, iteration):
    # Extrair as estatísticas ao longo das gerações
    gen = logbook.select('gen')
    avg = logbook.select('avg')
    std = logbook.select('std')
    min_ = logbook.select('min')
    max_ = logbook.select('max')

    plt.figure()
    plt.plot(gen, avg, label="average")
    plt.fill_between(gen, np.array(avg) - np.array(std), np.array(avg) + np.array(std), alpha=0.2)
    plt.plot(gen, min_, label="minimum")
    plt.plot(gen, max_, label="maximum")
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.legend(loc="lower right")
    plt.title(f"Fitness over Generations (Iteration {iteration})")
    plt.savefig(f'output/fitness/fitness_iteration_{iteration}.png')
    plt.close()


In [67]:

# Configurar MLflow
mlflow.set_experiment("Urban Vehicle Routing")


<Experiment: artifact_location='file:///home/pedro/Documents/BIA/p3/BD-AED2-final/AED2_roteamento_de_veiculos/code/mlruns/401334237198316218', creation_time=1721496467420, experiment_id='401334237198316218', last_update_time=1721496467420, lifecycle_stage='active', name='Urban Vehicle Routing', tags={}>

In [68]:

# Verifique se o diretório de saída existe e, se não, crie-o
if not os.path.exists('output'):
    os.makedirs('output')
if not os.path.exists('output/fitness'):
    os.makedirs('output/fitness')


In [69]:

location_points = [get_random_urban_location() for _ in range(10)]


In [70]:

for iteration, location_point in enumerate(location_points):
    with mlflow.start_run(run_name=f"Iteration_{iteration}"):
        graph_routes, best_individual, depot = run_genetic_algorithm(iteration, location_point)
        
        if graph_routes:
            plot_routes(G, graph_routes, depot, best_individual, iteration, location_point)
            
            # Registrar parâmetros e métricas no MLflow
            mlflow.log_param("iteration", iteration)
            mlflow.log_param("location", location_point)
            mlflow.log_param("num_vehicles", len(best_individual))
            mlflow.log_param("num_points", len(best_individual[0]) - 2)
            mlflow.log_metric("total_length", fitness(best_individual)[0])
            
            # Registrar o gráfico gerado
            mlflow.log_artifact(f'/home/pedro/Documents/BIA/p3/BD-AED2-final/output/iteration_{iteration}_location_{location_point}.png')
            mlflow.log_artifact(f'/home/pedro/Documents/BIA/p3/BD-AED2-final/output/fitness_iteration_{iteration}.png')
            
            # Salvar o melhor modelo (indivíduo)
            with open(f'/home/pedro/Documents/BIA/p3/BD-AED2-final/output/best_individual_{iteration}.pkl', 'wb') as f:
                pickle.dump(best_individual, f)
        else:
            print(f"Nenhuma solução encontrada para a localização {location_point} na iteração {iteration}")


/home/pedro/Documents/BIA/p3/BD-AED2-final/lib/python3.10/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


gen	nevals	avg	std	min    	max
0  	30    	inf	nan	16599.5	inf
1  	24    	inf	nan	16582.9	inf
2  	25    	17543.5	868.362	15551.7	19462.6
3  	22    	16806.4	933.716	15015.7	18304.8
4  	25    	15965.5	666.235	15015.7	17807.2
5  	22    	15532.4	626.495	13991  	17407  
6  	29    	14966.6	746.272	13160.4	15846.3
7  	23    	14501.5	1028.11	13160.4	16871  
8  	21    	13463.2	642.876	13160.4	15015.7
9  	27    	13188.1	149.093	13160.4	13991  
10 	26    	13160.4	1.81899e-12	13160.4	13160.4
11 	25    	13160.4	1.81899e-12	13160.4	13160.4
12 	27    	13160.4	1.81899e-12	13160.4	13160.4
13 	26    	13160.4	1.81899e-12	13160.4	13160.4
14 	30    	13160.4	1.81899e-12	13160.4	13160.4
15 	23    	13160.4	1.81899e-12	13160.4	13160.4
16 	26    	13160.4	1.81899e-12	13160.4	13160.4
17 	24    	13160.4	1.81899e-12	13160.4	13160.4
18 	20    	13160.4	1.81899e-12	13160.4	13160.4
19 	27    	13160.4	1.81899e-12	13160.4	13160.4
20 	23    	13160.4	1.81899e-12	13160.4	13160.4
21 	23    	13160.4	1.81899e-12	13160.4	13160.4

FileNotFoundError: [Errno 2] No such file or directory: '/home/pedro/Documents/BIA/p3/BD-AED2-final/output/iteration_0_location_(-3.7172, -38.5434).png'

---

### Load Model

In [ ]:
import pickle


In [ ]:

def load_model(file_path):
    with open(file_path, 'rb') as f:
        model = pickle.load(f)
    return model

def evaluate_model(model, G):
    total_length = fitness(model)[0]
    print(f"Total length of the route: {total_length}")


In [ ]:
# Carregar o modelo salvo (substitua o caminho pelo arquivo desejado)
model_file_path = 'output/best_individual_0.pkl'
best_individual = load_model(model_file_path)

# Avaliar o modelo carregado (assegure-se de usar o mesmo grafo G usado durante o treinamento)
location_point = (-16.681, -49.256)  # Exemplo: Goiânia
G = get_graph_from_location(location_point)
evaluate_model(best_individual, G)
